In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Load Titanic dataset
df = sns.load_dataset('titanic')

In [ ]:
# Fill missing numeric values with median
for col in df.select_dtypes(include='number').columns:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
# Fill missing categorical values with mode
for col in df.select_dtypes(include=['category', 'object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

In [ ]:
# Separate features and target
X = df.drop('survived', axis=1)
y = df['survived']

In [ ]:
# Identify categorical and numeric columns
categorical_cols = X.select_dtypes(include=['category', 'object']).columns
numeric_cols = X.select_dtypes(include='number').columns

In [ ]:
# One-hot encode categorical columns
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_cat = encoder.fit_transform(X[categorical_cols])
X_cat


array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 1., 0., ..., 1., 0., 0.]])

sparse_output=True → if you have huge datasets with many categories (memory saving).

sparse_output=False → if your data is small/medium and you want an easy-to-handle normal array.

fit → learns all the unique words in each categorical column.

transform → turns those words into numbers (0s and 1s).

fit_transform → does both at once.

"Take all my categorical (word) columns, turn them into numbers using One-Hot Encoding, and store the result in X_cat."

In [ ]:
# Get encoded feature names
encoded_cat_names = encoder.get_feature_names_out(categorical_cols)
encoded_cat_names

array(['sex_male', 'embarked_Q', 'embarked_S', 'class_Second',
       'class_Third', 'who_man', 'who_woman', 'deck_B', 'deck_C',
       'deck_D', 'deck_E', 'deck_F', 'deck_G', 'embark_town_Queenstown',
       'embark_town_Southampton', 'alive_yes'], dtype=object)

In [ ]:
# Combine numeric and encoded categorical columns
X_combined = np.hstack([X[numeric_cols].values, X_cat])
X_combined

array([[ 3., 22.,  1., ...,  0.,  1.,  0.],
       [ 1., 38.,  1., ...,  0.,  0.,  1.],
       [ 3., 26.,  0., ...,  0.,  1.,  1.],
       ...,
       [ 3., 28.,  1., ...,  0.,  1.,  0.],
       [ 1., 26.,  0., ...,  0.,  0.,  1.],
       [ 3., 32.,  0., ...,  1.,  0.,  0.]])

In [ ]:
# Create final feature names list
final_feature_names = list(numeric_cols) + list(encoded_cat_names)
print("\nFinal Features After Encoding:")
print(final_feature_names)


Final Features After Encoding:
['pclass', 'age', 'sibsp', 'parch', 'fare', 'sex_male', 'embarked_Q', 'embarked_S', 'class_Second', 'class_Third', 'who_man', 'who_woman', 'deck_B', 'deck_C', 'deck_D', 'deck_E', 'deck_F', 'deck_G', 'embark_town_Queenstown', 'embark_town_Southampton', 'alive_yes']


In [ ]:
# Standardize all features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)
X_scaled

array([[ 0.82737724, -0.56573646,  0.43279337, ..., -0.30756234,
         0.61583843, -0.78927234],
       [-1.56610693,  0.66386103,  0.43279337, ..., -0.30756234,
        -1.62380254,  1.2669898 ],
       [ 0.82737724, -0.25833709, -0.4745452 , ..., -0.30756234,
         0.61583843,  1.2669898 ],
       ...,
       [ 0.82737724, -0.1046374 ,  0.43279337, ..., -0.30756234,
         0.61583843, -0.78927234],
       [-1.56610693, -0.25833709, -0.4745452 , ..., -0.30756234,
        -1.62380254,  1.2669898 ],
       [ 0.82737724,  0.20276197, -0.4745452 , ...,  3.25137334,
        -1.62380254, -0.78927234]])

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:
# Create and train SVM model
svm_model = SVC(kernel='rbf', C=1, gamma='scale')
svm_model.fit(X_train, y_train)


SVC(C=1)

SVC = Support Vector Classifier (the SVM algorithm for classification).
Inside the brackets, you’re giving it settings (hyperparameters) to control how it learns.

 👉 Let’s break them:

kernel='rbf'
Kernel = "how to draw the boundary (line/curve) that separates the classes".

'rbf' means Radial Basis Function (a curved boundary).
Think of it like drawing wavy lines instead of straight ones to separate "Depressed" vs "Not Depressed".

C=1
C controls how strict the model is about mistakes.
Small C (like 0.1): model is relaxed, allows some mistakes, makes smoother boundary.
Large C (like 100): model is strict, tries to classify every point correctly, but might overfit.
Here, C=1 is a balanced choice.

gamma='scale'

Gamma controls how far the influence of one training point goes.
Low gamma → wider influence (boundary looks smoother).
High gamma → small influence (boundary hugs data tightly, might overfit).

scale' = let scikit-learn pick a good default value based on your data.

Two cases:
Low gamma (big circle 🍩)
Each point influences a large area.
Decision boundary (line/curve) is smooth
Model is less sensitive → may miss small patterns.

High gamma (tiny circle 🎯)
Each point influences only its close neighbors.
Decision boundary becomes wiggly (fits tightly).
Model is very sensitive → can overfit.

In [ ]:
# Predictions
y_pred = svm_model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.659217877094972

Classification Report:
               precision    recall  f1-score   support

           0       0.64      0.94      0.76       105
           1       0.76      0.26      0.38        74

    accuracy                           0.66       179
   macro avg       0.70      0.60      0.57       179
weighted avg       0.69      0.66      0.61       179



HYPER PARAMETER TUNNING:
Hyperparameters are the settings/knobs we choose before training the model.

C → how strict or flexible the margin is.
gamma → how far the influence of each point goes.

kernel → the type of curve/line (linear, rbf, poly, etc.).


Why tune hyperparameters?
Default values (like C=1, gamma='scale') are good,

but they may not be best for your dataset.

Tuning = trying different settings to find the best combination for accuracy.

Hyperparameter tuning is like choosing the best recipe 🍲:

C = how much spice you want.

gamma = how wide the flavor spreads.

kernel = the cooking style.

Grid Search = cooking every possible recipe and seeing which tastes best.
Random Search = cooking a few random recipes to save time.

In [ ]:
# Hyperparameter grid for SVM
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 0.01, 0.1, 1],
    'kernel': ['rbf', 'poly', 'sigmoid']
}

# param_grid = {
#     'C': [0.1, 1, 10, 100],        # How strict the margin is
#     'gamma': ['scale', 0.01, 0.1, 1],  # How far influence of each point goes
#     'kernel': ['rbf', 'poly', 'sigmoid']  # What type of curve/line to draw
# }
# 👉 This means:
# Try different C values
# Try different gamma values
# Try different kernels
# So total combinations = 4 × 4 × 3 = 48 models to check.

# Grid search with cross-validation
grid_search = GridSearchCV(
    SVC(),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# grid_search = GridSearchCV(
#     SVC(),              # Model to tune
#     param_grid,         # Menu of options
#     cv=5,               # 5-fold cross validation
#     scoring='accuracy', # Compare using accuracy
#     n_jobs=-1,          # Use all CPU cores (faster)
#     verbose=1           # Show progress
# )
# 📌 This will train 48 models × 5 folds = 240 trainings (!).
# But computer handles it 😎

grid_search.fit(X_train, y_train)
# The computer tests all combinations → finds best.

# Best parameters
print("Best Parameters:", grid_search.best_params_)
# Gives you the winning recipe 🏆 (best C, gamma, kernel).

# Evaluate with best model
best_model = grid_search.best_estimator_ # Best SVM model
y_pred = best_model.predict(X_test)  # Predictions

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Accuracy = how many predictions correct.
# Classification report = precision, recall, f1 for each class.

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'poly'}

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        74

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179



In [ ]:
# Checking my model with above results values that are best fits
svm_model = SVC(kernel='poly', C=1, gamma=0.01)
svm_model.fit(X_train, y_train)

SVC(C=1, gamma=0.01, kernel='poly')

In [ ]:
# Predictions
y_pred = svm_model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       105
           1       1.00      1.00      1.00        74

    accuracy                           1.00       179
   macro avg       1.00      1.00      1.00       179
weighted avg       1.00      1.00      1.00       179

